# sprint アンサンブル学習

# 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。

In [253]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import random

In [254]:
df = pd.read_csv("../example_input/house_prices/train.csv",
                 usecols=["GrLivArea", "YearBuilt", "SalePrice"])
df.head()

,YearBuilt,GrLivArea,SalePrice
0,2003,1710,208500
1,1976,1262,181500
2,2001,1786,223500
3,1915,1717,140000
4,2000,2198,250000


In [316]:
# 対数変換
df = np.log(df)

# 分割
X = df[["YearBuilt", "GrLivArea"]]
y = df[["SalePrice"]]
# ndarray変換
X_array = X.values
y_array = y.values
# 分割、シャフルTrue
X_train, X_valid, y_train, y_valid = train_test_split(
    X_array, y_array, test_size=0.2, random_state=0, shuffle=True)

# 各特徴量の標準化
# X、yを分けてインスタンス化
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_valid = scaler_X.transform(X_valid)
y_train = scaler_y.fit_transform(y_train).ravel()
y_valid = scaler_y.transform(y_valid).ravel()

# y_train = y_train.ravel()
# y_valid = y_valid.ravel()

## 単体での結果

In [317]:
# LinearRegression
reg_b = LinearRegression().fit(X_train, y_train)
pred_reg_b = reg_b.predict(X_valid)
print("MSE : LinearRegression")
print('{:.1e}'.format(mean_squared_error(y_valid, pred_reg_b)))  

MSE : LinearRegression
2.8e-01


In [318]:
# SVR
svr_b = SVR().fit(X_train, y_train)
pred_svr_b = svr_b.predict(X_valid)
print("MSE : SVR")
print('{:.1e}'.format(mean_squared_error(y_valid, pred_svr_b)))  

MSE : SVR
2.3e-01


In [319]:
# DecisionTreeRegressor
dt_b = DecisionTreeRegressor().fit(X_train, y_train)
pred_dt_b = dt_b.predict(X_valid)
print("MSE : DecisionTreeRegressor")
print('{:.1e}'.format(mean_squared_error(y_valid, pred_dt_b))) 

MSE : DecisionTreeRegressor
4.7e-01


# 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

In [320]:
# ブレンディング（平均）

# LinearRegression + SVR
pred_reg_svr = (pred_reg_b + pred_svr_b)/2
# LinearRegression + DecisionTreeRegressor
pred_reg_dt = (pred_reg_b + pred_dt_b)/2
# SVR + DecisionTreeRegressor
pred_svr_dt = (pred_svr_b + pred_dt_b)/3

print("MSE")
print("LinearRegression + SVR : ")
print('{:.1e}\n'.format(mean_squared_error(y_valid, pred_reg_svr))) 
print("LinearRegression + DecisionTreeRegressor : ")
print('{:.1e}\n'.format(mean_squared_error(y_valid, pred_reg_dt))) 
print("SVR + DecisionTreeRegressor : ")
print('{:.1e}'.format(mean_squared_error(y_valid, pred_svr_dt))) 

MSE
LinearRegression + SVR : 
2.5e-01

LinearRegression + DecisionTreeRegressor : 
3.0e-01

SVR + DecisionTreeRegressor : 
3.2e-01


# 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [321]:
# サンプル数を確認
X_train.shape[0]

1168

In [323]:
# サンプルのインデックスから重複ありでランダムサンプリング　500個
index_list = [i for i in range(X_train.shape[0])]

# 予測値の初期化
pred = 0
# 繰り返し回数10回
for n in range(10):
    index = random.choices(index_list, k=500)
    # DecisionTreeRegressor
    dt = DecisionTreeRegressor().fit(X_train[index], y_train[index])
    pred_dt = dt.predict(X_valid)
    # 合算
    pred += pred_dt

# 平均を出力
print("MSE : DecisionTreeRegressor / bagging")
print('{:.1e}'.format(mean_squared_error(y_valid, pred/10))) 

MSE : DecisionTreeRegressor / bagging
2.9e-01


# 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [448]:
# クロスバリデーション 学習
# K-foldごとスクラッチ

# 分割数
N = 4
# 分割後のサンプル数
num = X_train.shape[0]//N

# 学習モデルのリスト
model_list = [LinearRegression, SVR, DecisionTreeRegressor]
# 学習済みモデルの取得用リスト
model_para= []
model_para_all = []
# ブレンドデータを格納する空データ
blend_data = np.zeros([y_train.shape[0], 3])

for i, model in enumerate(model_list):
    # モデル毎リストの初期化
    model_para = []
    for n in range(N):
        # train_test_splitでシャッフルしてるから、ここでのシャッフル無し
        # 分割後のインデックスを取得
        index_valid = [a for a in range(num*n, num*(n+1))]
        index_train = [i for i in index_list if i not in index_valid]
        # 学習
        model_reg = model().fit(X_train[index_train], y_train[index_train])
        # 予測
        blend_data[index_valid, i] = model_reg.predict(X_train[index_valid])
        # モデルの取得
        model_para.append(model_reg)
    model_para_all.append(model_para)

print(blend_data.shape)

(1168, 3)


/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/user/anaconda3/lib/python3.7/site-packages/sklearn/

In [449]:
# ブレンドデータに対する学習
svr_st = SVR(C=1.0, epsilon=0.2, kernel="linear").fit(blend_data, y_train)

In [450]:
# クロスバリデーション 予測

# 予測値の初期化
blend_pred = np.zeros([y_valid.shape[0], 3], dtype=np.float)
# 予測
for n, models in enumerate(model_para_all):
    # モデル毎の予測値の初期化
    pred = np.zeros_like(y_valid)
    for model in models:
        pred = pred +  model.predict(X_valid)
    blend_pred[:, n] = pred/3

In [451]:
# ブレンドデータに対する予測
print("MSE : stacking")
print('{:.1e}'.format(mean_squared_error(y_valid, svr_st.predict(blend_pred))))  

MSE : stacking
2.9e-01


* スタッキングしてもあまり良くならない  
→決定木の深さを決めずに実行しているからか学習してる？